# Notebook Setup

In [ ]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"

In [ ]:
if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Import Modules

In [ ]:
# install required modules quietly
required_packages = ['geopandas', 'pyspark', 'azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

In [ ]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import math
import json

from pyspark.sql.functions import col
import pyspark.pandas as pd
from calendar import monthrange
from datetime import datetime
from io import BytesIO

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
if IN_COLLAB:
  os.chdir(MY_HOME_ABS_PATH)
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").config(
    "spark.jars.packages", 
    "org.apache.hadoop:hadoop-azure:3.3.1,com.microsoft.azure:azure-storage:8.6.6"
    ).getOrCreate()
# Check Spark Session Information
spark

# Define Constants

In [ ]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = tmp_dir
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/CO2_flux_gpp_modeling/DS_capstone_23Spring_CO2/Data/half_hourly_data"

site_metadata_filename = data_dir + os.sep + 'site-metadata.csv'

In [ ]:
# File
container = "baseline-data"
ext = "parquet"
ver = "0"
blob_name_base = f"baseline_all_v_{ver}"
train_blob_name_base = f"baseline-train-v-{ver}"
test_blob_name_base = f"baseline-test-v-{ver}"

In [ ]:
# "Golden" Sites
tier1_sites = ["US-MMS", "US-Vcp", "FR-Pue", "CH-Lae", "US-Var", "US-Ne2", "ES-LJu", "US-Ton"]
tier2_sites = ["US-UMB", "US-Me2", "FI-Hyy", "US-NR1", "IT-Lav", "US-Wkg", "US-ARM", "US-SRM"]

train_sites = tier1_sites + tier2_sites

# Selected Test Sites
test_sites = ["US-GLE", # ENF, Cold
              "US-AR1", # GRA, Temperate
              "US-Seg", # GRA, Arid
              "US-FR2", # WSA, Temperate
              "ES-LM2", # WSA, Arid
              "CA-Cbo", # DBF, Cold
              "FR-Lam", # CRO, Temperate
              "IT-Cpz", # EBF, Temperate
              "CN-Cha", # MF Cold
              "IT-Lsn", # OSH, Temperate
              ]

In [ ]:
# Define features and target variables of the data pipelines
included_features = ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA',
                     'datetime', 'year', 'month', 'day', 'hour', 'date',
                     'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 
                     'IGBP', 'koppen']
target_variable_qc = 'NEE_VUT_REF_QC'
target_variable = 'GPP_NT_VUT_REF'

# Get Gold Sample Site Data

In [ ]:
# Load site metadata
included_site_features = ['site_id', 'filename', 'elevation', 'lat', 'long',
                          'koppen_sub', 'koppen_main', 'koppen_name',
                          'c3c4', 'c4_percent']
site_metadata_df = pd.read_csv(site_metadata_filename, usecols = included_site_features)

# only focus on target sites
site_metadata_df = site_metadata_df.loc[site_metadata_df['site_id'].isin(train_sites + test_sites)]
print(f"size:{site_metadata_df.shape}")
site_metadata_df.reset_index(inplace=True, drop=True)
site_metadata_df

size:(26, 10)


,site_id,elevation,lat,long,koppen_sub,koppen_main,koppen_name,c3c4,c4_percent,filename
0,CN-Cha,NaN,42.40250,128.09580,22,4,Dwb,C3,12.17000,data_full_half_hourly_raw_v0_1_CN-Cha.csv
1,FR-Pue,270.00000,43.74130,3.59570,8,3,Csa,C3,6.59000,data_full_half_hourly_raw_v0_1_FR-Pue.csv
2,IT-Cpz,68.00000,41.70525,12.37611,8,3,Csa,C3,0.00000,data_full_half_hourly_raw_v0_1_IT-Cpz.csv
3,US-GLE,3197.00000,41.36653,-106.23990,27,4,Dfc,C3,0.16000,data_full_half_hourly_raw_v0_1_US-GLE.csv
4,US-NR1,3050.00000,40.03290,-105.54640,27,4,Dfc,C3,0.35000,data_full_half_hourly_raw_v0_1_US-NR1.csv
5,US-Ne2,362.00000,41.16487,-96.47010,25,4,Dfa,rotation,48.91000,NaN
6,US-SRM,1120.00000,31.82140,-110.86610,6,2,BSh,C3,55.39000,data_full_half_hourly_raw_v0_1_US-SRM.csv
7,US-Ton,177.00000,38.43160,-120.96598,8,3,Csa,C3,0.00000,data_full_half_hourly_raw_v0_1_US-Ton.csv
8,US-Var,129.00000,38.41330,-120.95070,8,3,Csa,C3,0.00000,data_full_half_hourly_raw_v0_1_US-Var.csv
9,US-Wkg,1531.00000,31.73650,-109.94190,7,2,BSk,C4,40.93000,data_full_half_hourly_raw_v0_1_US-Wkg.csv


# Stage 1: Trim and Merge Site Metadata

In [ ]:
all_features = ['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen']

In [ ]:
# Initial data clean and feature selections from raw data
data_df = data_cleanup(raw_data_dir, site_metadata_df[['site_id','filename']],
                  target_variable, target_variable_qc,
                  included_features)
print(f"Data size after cleanup: {data_df.shape}")

# Merge with site metadata
data_df = merge_site_metadata(data_df, site_metadata_df.drop(['filename', 'koppen_main', 'koppen_name'], axis=1))
print(f"Data size after after merged with site metadata: {data_df.shape}")

# Drop rows with NA
check_and_drop_na(data_df)
print(f"Data size after after final drop: {data_df.shape}")

#reorder columns
features = data_df.columns.to_list()
features.remove(target_variable)
data_df = data_df[([target_variable] + features)]

display(data_df.head())

CN-Cha: (16228, 27)
FR-Pue: (117200, 27)
IT-Cpz: (59175, 27)
US-GLE: (54687, 27)
US-NR1: (98652, 27)

ERROR: US-Ne2 is mssing hourly data.
US-SRM: (95419, 27)
US-Ton: (113031, 27)
US-Var: (119950, 27)
US-Wkg: (93319, 27)
CA-Cbo: (79273, 27)
US-AR1: (28956, 27)
US-ARM: (125756, 27)
US-FR2: (30426, 27)

ERROR: US-MMS is mssing hourly data.
US-Me2: (99780, 27)
US-Seg: (91884, 27)
US-UMB: (70639, 27)
US-Vcp: (78491, 27)
CH-Lae: (112718, 27)
ES-LJu: (112724, 27)
ES-LM2: (58806, 27)
FI-Hyy: (127362, 27)
FR-Lam: (115812, 27)
IT-Lav: (120885, 27)
IT-Lsn: (40182, 27)
Data size after cleanup: (2061355, 27)
Data size after after merged with site metadata: (2061355, 33)
Data has NA.


,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,GPP_NT_VUT_REF,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,c3c4,c4_percent
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16228,0,0,0,0,0


Data size after after final drop: (2045127, 33)


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,c3c4,c4_percent
16228,-0.53574,5.31100,25.01600,272.21800,1.70800,0.00000,97.93900,2001-01-01 08:30:00,2001,1,1,8,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000
16229,0.86438,5.74400,59.73400,272.21800,1.73800,0.00000,97.93900,2001-01-01 09:00:00,2001,1,1,9,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000
16230,-0.02627,6.17600,91.23500,272.21800,1.76700,0.00000,97.93900,2001-01-01 09:30:00,2001,1,1,9,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000
16231,-0.17229,6.60800,79.26400,333.93300,1.79700,0.05000,97.93900,2001-01-01 10:00:00,2001,1,1,10,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000
16232,1.20865,7.04300,94.92900,333.93300,1.81700,0.00000,97.92300,2001-01-01 10:30:00,2001,1,1,10,2001-01-01,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000


In [ ]:
data_df.describe()

,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,minute,elevation,lat,long,koppen_sub,c4_percent
count,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000,2045127.00000
mean,5.87229,15.18393,376.25804,319.49078,10.11018,0.03958,91.62118,2010.10650,6.55024,15.75130,11.92300,0.29796,0.52726,0.13143,0.08435,0.24830,0.04657,0.07608,0.26551,0.20770,0.12646,15.00078,901.20832,41.38344,-58.39359,15.86035,7.39929
std,7.61167,9.17084,267.25693,49.30580,9.61124,0.19043,9.25367,4.65177,3.00549,8.81239,3.75671,0.13328,0.22261,0.07261,0.06102,0.06284,0.04531,0.04748,0.06925,0.09472,0.08461,15.00000,906.36713,7.03857,57.53619,8.51506,14.29274
min,-49.73720,-30.68700,0.00100,107.63900,0.00000,0.00000,66.03100,2001.00000,1.00000,1.00000,3.00000,-0.11958,-0.18252,-0.01764,0.00540,0.03050,0.00000,0.00000,0.00000,0.01320,0.00000,0.00000,1.00000,29.94950,-121.55740,5.00000,0.00000
25%,0.44478,8.97400,138.39100,285.18600,3.23000,0.00000,85.19600,2007.00000,4.00000,8.00000,9.00000,0.20223,0.33970,0.07913,0.03670,0.20250,0.02110,0.04440,0.20950,0.13000,0.05590,0.00000,180.00000,36.60580,-109.94190,8.00000,0.00000
50%,3.14854,15.31700,343.34100,321.12900,6.98900,0.00000,97.16200,2010.00000,7.00000,16.00000,12.00000,0.28931,0.55534,0.11931,0.06750,0.23890,0.03570,0.06640,0.27080,0.19530,0.10880,30.00000,314.00000,40.03290,-97.48880,14.00000,0.35000
75%,9.46059,21.72100,585.40800,354.95800,13.69700,0.00000,99.03900,2014.00000,9.00000,23.00000,15.00000,0.36828,0.70229,0.16489,0.11910,0.28780,0.05610,0.09460,0.32010,0.28260,0.18360,30.00000,1531.00000,44.45230,3.59570,26.00000,6.59000
max,85.03090,44.08700,1094.34100,473.08500,80.09100,15.49300,103.92100,2020.00000,12.00000,31.00000,23.00000,2.38835,0.93551,0.42385,0.81200,0.77620,0.76890,0.80530,0.46660,0.47920,0.43470,30.00000,3197.00000,61.84741,24.29477,27.00000,55.39000


In [ ]:
data_df.site_id.unique()

array(['FR-Pue', 'IT-Cpz', 'US-GLE', 'US-NR1', 'US-SRM', 'US-Ton',
       'US-Var', 'US-Wkg', 'CA-Cbo', 'US-AR1', 'US-ARM', 'US-FR2',
       'US-Me2', 'US-Seg', 'US-UMB', 'US-Vcp', 'CH-Lae', 'ES-LJu',
       'ES-LM2', 'FI-Hyy', 'FR-Lam', 'IT-Lav', 'IT-Lsn'], dtype=object)

In [ ]:
data_df.IGBP.unique()

array(['EBF', 'ENF', 'WSA', 'GRA', 'DBF', 'CRO', 'MF', 'OSH'],
      dtype=object)

In [ ]:
data_df.koppen.unique()

array(['Temperate', 'Cold', 'Arid'], dtype=object)

# CheckPoint: Upload Data to Azure Storage Blob as Parquet

In [ ]:
# Upload to Azure Storage Blob
# ref: https://stackoverflow.com/a/54666079
data_cleanup_checkpoint = True
tag = "raw"
blob_name = f"{blob_name_base}_{tag}.{ext}"

if data_cleanup_checkpoint:

  parquet_file = BytesIO()
  data_df.to_parquet(parquet_file, engine='pyarrow')
  parquet_file.seek(0)

  azStorageClient = AzStorageClient(az_cred_file)
  azStorageClient.uploadBlob(container, blob_name, parquet_file, overwrite=True)

File uploaded to baseline-data/baseline_all_v_0_raw.parquet


# Stage 2: Data Transform - Convert to Model Ready Data
- Encode data
- Split into train and test
- Assemble data through VectorAssembler
- Normalized data through minmax

In [ ]:
load_data_from_previous_checkpoint = True
useSpark = False

if load_data_from_previous_checkpoint:
  data_df = None
  print(f"loading {tmp_dir + os.sep + blob_name}...")
  if not (os.path.exists(tmp_dir + os.sep + blob_name)):
      if not (os.path.exists(tmp_dir)):
          os.mkdir(tmp_dir)
      azStorageClient = AzStorageClient(az_cred_file)
      file_stream = azStorageClient.downloadBlob2Stream(container, blob_name)
      data_df = pd.read_parquet(file_stream, engine='pyarrow')
      data_df.to_parquet(tmp_dir + os.sep + blob_name)
  
  if useSpark:
    data_df = spark.read.parquet(tmp_dir + os.sep + blob_name)
    data_df = data_df.drop(*['__index_level_0__'])
    print(f"Data loaded: {data_df.count()} rows x {len(data_df.columns)} columns.")
  else:
    data_df = pd.read_parquet(tmp_dir + os.sep + blob_name, engine='pyarrow')
    print(f"Data size: {data_df.shape}.")


loading /content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling/.tmp/baseline_all_v_0_raw.parquet...
Data size: (2045127, 33).


In [ ]:
# Drop 'datetime', 'date'; as they are already represented by other columns

categorical_cols = ['IGBP', 'c3c4', 'koppen_sub', 'koppen']

if useSpark:
  data_df = data_df.drop(*['datetime', 'date'])
else:
  data_df = data_df.drop(['datetime', 'date'], axis = 1)

In [ ]:
if useSpark:
  from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
  string_indexer = StringIndexer(inputCols=categorical_cols, outputCols=[x + "_Index" for x in categorical_cols]) 
  data_df = string_indexer.fit(data_df).transform(data_df)

  one_hot_encoder  = OneHotEncoder(inputCols=string_indexer.getOutputCols(), outputCols=[x + "_OHE" for x in categorical_cols])
  data_df = one_hot_encoder.fit(data_df).transform(data_df)

  data_df = data_df.drop(*string_indexer.getOutputCols())

  print(f"Data size after encoding: {data_df.count()} rows x {len(data_df.columns)} columns.")
  data_df.show()
else:
  from sklearn.preprocessing import LabelEncoder
  # One-hot encoding
  # encoded_df = pd.get_dummies(data_df[categorical_cols],
  #                             columns=categorical_cols, 
  #                             dummy_na=False)
  encoded_df = data_df[categorical_cols].apply(LabelEncoder().fit_transform)
  encoded_df = encoded_df.add_suffix('_nbr')
  data_df = data_df.join(encoded_df)
  print(f"Data size after encoding: {data_df.shape}")
  display(data_df.head())


Data size after encoding: (2045127, 35)


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,c3c4,c4_percent,IGBP_nbr,c3c4_nbr,koppen_sub_nbr,koppen_nbr
16228,-0.53574,5.31100,25.01600,272.21800,1.70800,0.00000,97.93900,2001,1,1,8,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000,2,0,3,2
16229,0.86438,5.74400,59.73400,272.21800,1.73800,0.00000,97.93900,2001,1,1,9,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000,2,0,3,2
16230,-0.02627,6.17600,91.23500,272.21800,1.76700,0.00000,97.93900,2001,1,1,9,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000,2,0,3,2
16231,-0.17229,6.60800,79.26400,333.93300,1.79700,0.05000,97.93900,2001,1,1,10,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000,2,0,3,2
16232,1.20865,7.04300,94.92900,333.93300,1.81700,0.00000,97.92300,2001,1,1,10,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000,2,0,3,2


In [ ]:
# Get features
features = data_df.columns
if useSpark:
  features = data_df.columns
else:
  features = data_df.columns.to_list()

features.remove(target_variable)
features.remove('site_id')

for f in categorical_cols:
  features.remove(f)
print(f"Features({len(features)}): {features}")

Features(29): ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA', 'year', 'month', 'day', 'hour', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'minute', 'elevation', 'lat', 'long', 'c4_percent', 'IGBP_nbr', 'c3c4_nbr', 'koppen_sub_nbr', 'koppen_nbr']


In [ ]:
# Assemble data
if useSpark:
  assembler = VectorAssembler(inputCols=features, outputCol="vectorized_features")
  data_df = assembler.transform(data_df)

  print(f"Data size after assembling: {data_df.count()} rows x {len(data_df.columns)} columns.")
  data_df.show()

In [ ]:
# Split into train and test sets
if useSpark:
  train_df = data_df.filter(col('site_id').isin(train_sites))
  test_df = data_df.filter(col('site_id').isin(test_sites))
  print(f"Train data size: {train_df.count()} rows x {len(train_df.columns)} columns.")
  print(f"Test data size: {test_df.count()} rows x {len(test_df.columns)} columns.")

  print("Train data peak:")
  train_df.show(5, False)
  print("Test data peak:")
  test_df.show(5, False)
else:
  train_df = data_df[data_df['site_id'].isin(train_sites)]
  test_df  = data_df[data_df['site_id'].isin(test_sites)]
  print(f"Train data size: {train_df.shape}.")
  print(f"Test data size: {test_df.shape}.")

  print("Train data peak:")
  display(train_df.head(5))
  print("Test data peak:")
  display(test_df.head(5))

Train data size: (1485926, 35).
Test data size: (559201, 35).
Train data peak:


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,c3c4,c4_percent,IGBP_nbr,c3c4_nbr,koppen_sub_nbr,koppen_nbr
16228,-0.53574,5.31100,25.01600,272.21800,1.70800,0.00000,97.93900,2001,1,1,8,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000,2,0,3,2
16229,0.86438,5.74400,59.73400,272.21800,1.73800,0.00000,97.93900,2001,1,1,9,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000,2,0,3,2
16230,-0.02627,6.17600,91.23500,272.21800,1.76700,0.00000,97.93900,2001,1,1,9,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000,2,0,3,2
16231,-0.17229,6.60800,79.26400,333.93300,1.79700,0.05000,97.93900,2001,1,1,10,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,0,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000,2,0,3,2
16232,1.20865,7.04300,94.92900,333.93300,1.81700,0.00000,97.92300,2001,1,1,10,0.24998,0.73349,0.10592,0.02220,0.14440,0.00740,0.02670,0.14860,0.09770,0.00000,EBF,Temperate,30,FR-Pue,270.00000,43.74130,3.59570,8,C3,6.59000,2,0,3,2


Test data peak:


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,c3c4,c4_percent,IGBP_nbr,c3c4_nbr,koppen_sub_nbr,koppen_nbr
133428,2.19090,4.66000,5.44900,218.92000,2.90400,0.00000,100.76500,2001,1,1,7,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,30,IT-Cpz,68.00000,41.70525,12.37611,8,C3,0.00000,2,0,3,2
133429,2.83647,5.08600,71.31800,218.92000,2.95200,0.00000,100.78900,2001,1,1,8,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,0,IT-Cpz,68.00000,41.70525,12.37611,8,C3,0.00000,2,0,3,2
133430,4.03906,5.51100,132.32000,218.92000,3.00000,0.00000,100.81300,2001,1,1,8,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,30,IT-Cpz,68.00000,41.70525,12.37611,8,C3,0.00000,2,0,3,2
133431,7.79772,5.93700,187.41100,218.92000,3.04800,0.00000,100.83700,2001,1,1,9,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,0,IT-Cpz,68.00000,41.70525,12.37611,8,C3,0.00000,2,0,3,2
133432,6.93281,6.36200,235.65000,218.92000,3.09600,0.00000,100.86100,2001,1,1,9,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,30,IT-Cpz,68.00000,41.70525,12.37611,8,C3,0.00000,2,0,3,2


In [ ]:
train_df.describe()

,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,minute,elevation,lat,long,koppen_sub,c4_percent,IGBP_nbr,c3c4_nbr,koppen_sub_nbr,koppen_nbr
count,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000,1485926.00000
mean,5.86211,14.87724,377.56323,316.15936,10.11574,0.03757,90.79849,2009.75172,6.53593,15.74438,11.89376,0.29739,0.53815,0.13063,0.07894,0.24123,0.04288,0.07109,0.25797,0.20036,0.11929,14.99925,971.07777,41.75447,-62.35813,16.51431,8.82611,3.65611,0.23206,4.40883,1.26914
std,7.52193,9.04516,268.53741,47.26856,9.61817,0.18091,8.84375,4.52259,2.98991,8.81076,3.77390,0.13132,0.21932,0.07137,0.05540,0.06086,0.03907,0.04160,0.06960,0.09369,0.07861,15.00001,866.88847,7.77408,59.15444,8.71496,15.99510,1.99361,0.58947,1.85470,0.67137
min,-49.73720,-29.74000,0.00100,142.77000,0.00000,0.00000,67.40500,2001.00000,1.00000,1.00000,3.00000,-0.11958,-0.18252,-0.01715,0.00540,0.03050,0.00000,0.00000,0.00000,0.01320,0.00000,0.00000,129.00000,31.73650,-121.55740,6.00000,0.00000,0.00000,0.00000,1.00000,0.00000
25%,0.45157,8.69200,138.70625,283.86900,3.18000,0.00000,85.24200,2006.00000,4.00000,8.00000,9.00000,0.20336,0.34150,0.08147,0.03370,0.19840,0.01960,0.04220,0.20210,0.11890,0.05150,0.00000,234.00000,36.60580,-110.86610,8.00000,0.00000,3.00000,0.00000,3.00000,1.00000
50%,3.23428,14.93100,344.52850,317.35000,6.98700,0.00000,93.16700,2010.00000,7.00000,16.00000,12.00000,0.28756,0.56813,0.11822,0.06390,0.23080,0.03400,0.06320,0.26110,0.18680,0.10330,0.00000,689.00000,40.03290,-97.48880,14.00000,0.04000,3.00000,0.00000,4.00000,1.00000
75%,9.48720,21.28200,587.46300,349.28900,13.73200,0.00000,98.71700,2013.00000,9.00000,23.00000,15.00000,0.36012,0.70597,0.15872,0.11660,0.27750,0.05450,0.09120,0.31520,0.27910,0.18240,30.00000,1531.00000,45.55980,3.59570,26.00000,10.72000,5.00000,0.00000,6.00000,2.00000
max,85.03090,42.58700,1094.34100,473.01100,75.68400,15.49300,103.38300,2020.00000,12.00000,31.00000,23.00000,2.38835,0.93551,0.42385,0.79710,0.77290,0.76890,0.78650,0.46660,0.42800,0.35730,30.00000,3050.00000,61.84741,24.29477,27.00000,55.39000,7.00000,2.00000,7.00000,2.00000


In [ ]:
test_df.describe()

,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,minute,elevation,lat,long,koppen_sub,c4_percent,IGBP_nbr,c3c4_nbr,koppen_sub_nbr,koppen_nbr
count,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000
mean,5.89935,15.99888,372.78984,328.34315,10.09542,0.04491,93.80726,2011.04924,6.58824,15.76970,12.00068,0.29945,0.49832,0.13357,0.09872,0.26706,0.05637,0.08935,0.28554,0.22718,0.14550,15.00486,715.54907,40.39754,-47.85888,14.12260,3.60790,3.05929,1.00171,3.60351,1.22150
std,7.84509,9.44850,263.79306,53.34800,9.59278,0.21359,9.93809,4.85439,3.04618,8.81670,3.70952,0.13835,0.22863,0.07576,0.07196,0.06416,0.05761,0.05835,0.06413,0.09469,0.09622,15.00001,979.83832,4.38791,51.53537,7.69401,6.89128,2.43910,1.25281,2.10843,0.84344
min,-49.38330,-30.68700,0.00100,107.63900,0.00000,0.00000,66.03100,2001.00000,1.00000,1.00000,4.00000,-0.09768,-0.02273,-0.01764,0.01480,0.09860,0.00500,0.01930,0.09130,0.02990,0.01230,0.00000,1.00000,29.94950,-106.70190,5.00000,0.00000,0.00000,0.00000,0.00000,0.00000
25%,0.42672,9.80400,137.52300,289.93600,3.35200,0.00000,84.79300,2007.00000,4.00000,8.00000,9.00000,0.19767,0.32305,0.07163,0.04730,0.21550,0.02510,0.05220,0.24010,0.15750,0.07070,0.00000,120.00000,36.42670,-106.23990,7.00000,0.24000,1.00000,0.00000,2.00000,0.00000
50%,2.93147,16.40000,340.39600,334.09000,6.99500,0.00000,98.73900,2011.00000,7.00000,16.00000,12.00000,0.29334,0.52381,0.12294,0.07760,0.26630,0.03980,0.07450,0.29010,0.21110,0.11770,30.00000,180.00000,41.70525,-79.93330,14.00000,1.09000,3.00000,0.00000,4.00000,1.00000
75%,9.38355,22.74500,580.10600,369.34900,13.61200,0.00000,99.85900,2015.00000,9.00000,23.00000,15.00000,0.38945,0.68621,0.18062,0.13040,0.31170,0.06460,0.10430,0.33200,0.29520,0.19270,30.00000,1622.00000,43.49644,1.23788,14.00000,3.10000,4.00000,2.00000,4.00000,2.00000
max,83.61300,44.08700,1050.62600,473.08500,80.09100,13.03500,103.92100,2020.00000,12.00000,31.00000,23.00000,1.41056,0.90239,0.38458,0.81200,0.77620,0.76400,0.80530,0.44180,0.47920,0.43470,30.00000,3197.00000,45.74048,12.75030,27.00000,27.54000,7.00000,3.00000,7.00000,2.00000


In [ ]:
# Normalize data
if useSpark:
  scaler = MinMaxScaler(inputCol='vectorized_features', outputCol='features')
  scaler_model = scaler.fit(train_df)
  train_df = scaler_model.transform(train_df)
  test_df = scaler_model.transform(test_df)

  train_df = train_df.drop(*['vectorized_features'])
  test_df = test_df.drop(*['vectorized_features'])
  print(f"Train data size: {train_df.count()} rows x {len(train_df.columns)} columns.")
  print(f"Test data size: {test_df.count()} rows x {len(test_df.columns)} columns.")

  print("Train data peak:")
  train_df.show(5, False)
  print("Test data peak:")
  test_df.show(5, False)
else:
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler().fit(train_df[features])
  train_df.loc[:,features] = scaler.transform(train_df[features])
  test_df.loc[:,features] = scaler.transform(test_df[features])
  print(f"Train data size: {train_df.shape}.")
  print(f"Test data size: {test_df.shape}.")

  print("Train data peak:")
  display(train_df.head(5))
  print("Test data peak:")
  display(test_df.head(5))


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


Train data size: (1485926, 35).
Test data size: (559201, 35).
Train data peak:


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,c3c4,c4_percent,IGBP_nbr,c3c4_nbr,koppen_sub_nbr,koppen_nbr
16228,-0.53574,0.48462,0.02286,0.39198,0.02257,0.00000,0.84869,0.00000,0.00000,0.00000,0.25000,0.14735,0.81931,0.27907,0.02122,0.15342,0.00962,0.03395,0.31847,0.20371,0.00000,EBF,Temperate,1.00000,FR-Pue,0.04827,0.39869,0.85808,8,C3,0.11897,0.28571,0.00000,0.33333,1.00000
16229,0.86438,0.49061,0.05458,0.39198,0.02296,0.00000,0.84869,0.00000,0.00000,0.00000,0.30000,0.14735,0.81931,0.27907,0.02122,0.15342,0.00962,0.03395,0.31847,0.20371,0.00000,EBF,Temperate,0.00000,FR-Pue,0.04827,0.39869,0.85808,8,C3,0.11897,0.28571,0.00000,0.33333,1.00000
16230,-0.02627,0.49658,0.08337,0.39198,0.02335,0.00000,0.84869,0.00000,0.00000,0.00000,0.30000,0.14735,0.81931,0.27907,0.02122,0.15342,0.00962,0.03395,0.31847,0.20371,0.00000,EBF,Temperate,1.00000,FR-Pue,0.04827,0.39869,0.85808,8,C3,0.11897,0.28571,0.00000,0.33333,1.00000
16231,-0.17229,0.50255,0.07243,0.57886,0.02374,0.00323,0.84869,0.00000,0.00000,0.00000,0.35000,0.14735,0.81931,0.27907,0.02122,0.15342,0.00962,0.03395,0.31847,0.20371,0.00000,EBF,Temperate,0.00000,FR-Pue,0.04827,0.39869,0.85808,8,C3,0.11897,0.28571,0.00000,0.33333,1.00000
16232,1.20865,0.50857,0.08674,0.57886,0.02401,0.00000,0.84824,0.00000,0.00000,0.00000,0.35000,0.14735,0.81931,0.27907,0.02122,0.15342,0.00962,0.03395,0.31847,0.20371,0.00000,EBF,Temperate,1.00000,FR-Pue,0.04827,0.39869,0.85808,8,C3,0.11897,0.28571,0.00000,0.33333,1.00000


Test data peak:


,GPP_NT_VUT_REF,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,c3c4,c4_percent,IGBP_nbr,c3c4_nbr,koppen_sub_nbr,koppen_nbr
133428,2.19090,0.47562,0.00498,0.23059,0.03837,0.00000,0.92723,0.00000,0.00000,0.00000,0.20000,0.16838,0.82976,0.32125,0.02400,0.18400,0.02341,0.03700,0.50407,0.20371,0.13546,EBF,Temperate,1.00000,IT-Cpz,-0.02088,0.33107,0.91828,8,C3,0.00000,0.28571,0.00000,0.33333,1.00000
133429,2.83647,0.48151,0.06517,0.23059,0.03900,0.00000,0.92790,0.00000,0.00000,0.00000,0.25000,0.16838,0.82976,0.32125,0.02400,0.18400,0.02341,0.03700,0.50407,0.20371,0.13546,EBF,Temperate,0.00000,IT-Cpz,-0.02088,0.33107,0.91828,8,C3,0.00000,0.28571,0.00000,0.33333,1.00000
133430,4.03906,0.48738,0.12091,0.23059,0.03964,0.00000,0.92857,0.00000,0.00000,0.00000,0.25000,0.16838,0.82976,0.32125,0.02400,0.18400,0.02341,0.03700,0.50407,0.20371,0.13546,EBF,Temperate,1.00000,IT-Cpz,-0.02088,0.33107,0.91828,8,C3,0.00000,0.28571,0.00000,0.33333,1.00000
133431,7.79772,0.49327,0.17125,0.23059,0.04027,0.00000,0.92923,0.00000,0.00000,0.00000,0.30000,0.16838,0.82976,0.32125,0.02400,0.18400,0.02341,0.03700,0.50407,0.20371,0.13546,EBF,Temperate,0.00000,IT-Cpz,-0.02088,0.33107,0.91828,8,C3,0.00000,0.28571,0.00000,0.33333,1.00000
133432,6.93281,0.49915,0.21533,0.23059,0.04091,0.00000,0.92990,0.00000,0.00000,0.00000,0.30000,0.16838,0.82976,0.32125,0.02400,0.18400,0.02341,0.03700,0.50407,0.20371,0.13546,EBF,Temperate,1.00000,IT-Cpz,-0.02088,0.33107,0.91828,8,C3,0.00000,0.28571,0.00000,0.33333,1.00000


# Checkpoint: Upload train and test to Azure Blob Storage

In [ ]:
final_checkpoint = True

if final_checkpoint:
  # Initialize AzStorageClient 
  azStorageClient = AzStorageClient(az_cred_file)
  
  if useSpark:
    sessionkeys = azStorageClient.getSparkSessionKeys()
    spark.conf.set(sessionkeys[0],sessionkeys[1])

    # Upload train dataset
    train_blob_path = f"wasbs://{container}@{sessionkeys[2]}.blob.core.windows.net/{train_blob_name_base}"
    print(f"Uploading train dataset to {train_blob_path}...")
    train_df.write.format("parquet").mode("overwrite").save(train_blob_path)

    # Upload test dataset
    test_blob_path = f"wasbs://{container}@{sessionkeys[2]}.blob.core.windows.net/{test_blob_name_base}"
    print(f"Uploading test dataset to {test_blob_path}...")
    test_df.write.format("parquet").mode("overwrite").save(test_blob_path)
  
  else:
    # Upload train dataset
    train_file = BytesIO()
    train_df.to_parquet(train_file, engine='pyarrow')
    train_file.seek(0)
    train_blob_name= f"{train_blob_name_base}.{ext}"
    print(f"Uploading train dataset to {train_blob_name}...")
    azStorageClient.uploadBlob(container, train_blob_name, train_file, overwrite=True)

    # Upload test dataset
    test_file = BytesIO()
    test_df.to_parquet(test_file, engine='pyarrow')
    test_file.seek(0)
    test_blob_name= f"{test_blob_name_base}.{ext}"
    print(f"Uploading test dataset to {test_blob_name}...")
    azStorageClient.uploadBlob(container, test_blob_name, test_file, overwrite=True)

Uploading train dataset to baseline-train-v-0.parquet...
File uploaded to baseline-data/baseline-train-v-0.parquet
Uploading test dataset to baseline-test-v-0.parquet...
File uploaded to baseline-data/baseline-test-v-0.parquet
